# Model 


# import data

In [1]:
import os
import sys
from pyprojroot import here

import pandas as pd

In [2]:
#pip list

In [3]:
# Read in the data in the previous step
df = pd.read_pickle('..\data\processed\data_clean.pkl')
len(df)

3134

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3134 entries, 0 to 3133
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   project_title                  3134 non-null   object        
 1   project_description            3134 non-null   object        
 2   area_primary                   3129 non-null   object        
 3   discipline_primary             3130 non-null   float64       
 4   approval_date                  3134 non-null   datetime64[ns]
 5   lead_research_institution      3134 non-null   object        
 6   city                           3133 non-null   object        
 7   ontario_commitment             3134 non-null   int32         
 8   total_project_costs            3134 non-null   int32         
 9   keyword                        1077 non-null   object        
 10  year                           3134 non-null   int64         
 11  month            

# Split the data to train-test

In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
y1=df['label_total_project_costs'] #multiclass classifier model1
y2=df['label_ontario_commitment'] #multiclass classifier model2
y3=df['total_project_costs'] #regression model3
y4=df['ontario_commitment']  # regression model 4
y5=df['ontario_share']       #regression model 5
y6=df['two_labela_ontario_commitment']
x=df['project_title']



In [7]:
def split_data(x,y):
    x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
    return (x_train,x_test,y_train,y_test)

In [8]:
x_train,x_test,y_train1,y_test1 =split_data(x,y1)
x_train,x_test,y_train2,y_test2 =split_data(x,y2)
x_train,x_test,y_train3,y_test3 =split_data(x,y3)
x_train,x_test,y_train4,y_test4 =split_data(x,y4)
x_train,x_test,y_train5,y_test5 =split_data(x,y5)
x_train,x_test,y_train6,y_test6 =split_data(x,y6)

In [9]:
len(x_train),len(x_test)

(2350, 784)

In [10]:
x_train.shape,y_train1.shape

((2350,), (2350,))

In [11]:
x_train

2855    system for flame synthesis collection and char...
1644    centre for the evaluation of technological inn...
2657    predictive immune monitoring approaches for pr...
2765                            design observation studio
518     biotechnology for wood fibre processing and en...
                              ...                        
3092    development of valueadded biodegradable multil...
1095    experimental facility for advanced manufacturi...
1130    neural and metabolic correlates of emotional p...
1294                      watershed analysis and modeling
860                    the ecological genomics laboratory
Name: project_title, Length: 2350, dtype: object

# Tf-idf


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Atieh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords

In [14]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',
                             lowercase=True,
                             max_df=0.9,
                             min_df=2,
                             ngram_range=(1,1),
                             stop_words='english')

In [15]:
tfidfvectorizer.fit(x_train)
tfidf_train = tfidfvectorizer.transform(x_train)

In [16]:
tfidf_test  = tfidfvectorizer.transform(x_test)

In [17]:
tfidf_train.shape,tfidf_test.shape

((2350, 1605), (784, 1605))

In [18]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
count_wm=countvectorizer.fit_transform(x_train)

In [19]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
df_countvect = pd.DataFrame(data = count_wm.toarray(),columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_train.toarray(),columns = tfidf_tokens)
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

Count Vectorizer

      1000mhz  120c  1861  1871  18752010  1891  1d  20  2020  2d  ...  york  \
0           0     0     0     0         0     0   0   0     0   0  ...     0   
1           0     0     0     0         0     0   0   0     0   0  ...     0   
2           0     0     0     0         0     0   0   0     0   0  ...     0   
3           0     0     0     0         0     0   0   0     0   0  ...     0   
4           0     0     0     0         0     0   0   0     0   0  ...     0   
...       ...   ...   ...   ...       ...   ...  ..  ..   ...  ..  ...   ...   
2345        0     0     0     0         0     0   0   0     0   0  ...     0   
2346        0     0     0     0         0     0   0   0     0   0  ...     0   
2347        0     0     0     0         0     0   0   0     0   0  ...     0   
2348        0     0     0     0         0     0   0   0     0   0  ...     0   
2349        0     0     0     0         0     0   0   0     0   0  ...     0   

      young  youth  y

In [21]:
tfidf_train.shape, tfidf_test.shape

((2350, 1605), (784, 1605))

In [22]:
len(tfidf_tokens)

1605

In [23]:
len(set(tfidf_tokens))

1605

In [24]:
#fit change and fit the model
#transform just return the value of known model
type(tfidf_train)

scipy.sparse.csr.csr_matrix

In [25]:
tfidf_tokens[100:]

['associated',
 'assurance',
 'asthma',
 'astronomical',
 'astronomy',
 'astrophysics',
 'asymmetric',
 'atmosphere',
 'atmospheric',
 'atomic',
 'atomicallyresolved',
 'atoms',
 'attention',
 'attosecond',
 'autism',
 'automated',
 'automation',
 'automotive',
 'autonomic',
 'autonomous',
 'avian',
 'aviation',
 'awareness',
 'background',
 'bacteria',
 'bacterial',
 'bacteriophages',
 'balance',
 'barriers',
 'based',
 'basic',
 'basis',
 'beamline',
 'behavior',
 'behavioral',
 'behaviors',
 'behaviour',
 'behavioural',
 'benefit',
 'best',
 'beta',
 'beverage',
 'big',
 'bilingual',
 'bioactive',
 'bioanalytical',
 'bioarchaeology',
 'biochemical',
 'biochemistry',
 'biodiversity',
 'bioengineering',
 'biofilm',
 'biofuel',
 'biogeochemical',
 'biogeochemistry',
 'bioimaging',
 'bioinformatics',
 'biological',
 'biology',
 'biomarker',
 'biomarkers',
 'biomass',
 'biomaterial',
 'biomaterials',
 'biomatrix',
 'biomechanical',
 'biomechanics',
 'biomedical',
 'biomedicine',
 'biomim

In [26]:
tfidf_train.nonzero()[1] #Return the indices of the elements that are non-zero.

array([1446,  993,  278, ...,  811,  628,  446], dtype=int32)

In [27]:
tfidf_train.nonzero()

(array([   0,    0,    0, ..., 2349, 2349, 2349], dtype=int32),
 array([1446,  993,  278, ...,  811,  628,  446], dtype=int32))

In [28]:
import numpy as np
a=np.array([[3,0,4],[4,7,0]])
a, a.nonzero()

(array([[3, 0, 4],
        [4, 7, 0]]),
 (array([0, 0, 1, 1], dtype=int64), array([0, 2, 0, 1], dtype=int64)))

In [29]:
a.nonzero()[1]

array([0, 2, 0, 1], dtype=int64)

In [30]:
tfidf_train.shape

(2350, 1605)

In [31]:
#extract the words for with the tf-idf of higher than 0.7 for train set.
m=0
d=[]
for col in tfidf_train.nonzero()[1]:
    if tfidf_train[tfidf_train.nonzero()[0][m],col] > 0.7:
        #print (words[col], '-', tfidf_train[tfidf_train.nonzero()[0][m],col])
        d.append((tfidf_tokens[col],tfidf_train[tfidf_train.nonzero()[0][m],col]))
        m=m+1
    else:
        m=m+1
df=pd.DataFrame(d, columns=('tfidf_tokens', 'tf-idf'))
df.sort_values(by=['tf-idf'],ascending=False)    

,tfidf_tokens,tf-idf
119,modulation,1.000000
31,attosecond,1.000000
328,infrastructure,1.000000
187,array,1.000000
267,computing,1.000000
...,...,...
330,learning,0.701984
308,adolescent,0.701680
13,leading,0.700922
136,public,0.700793


In [32]:
print(tfidf_train) 

  (0, 1446)	0.43193348220168404
  (0, 993)	0.5726629431746175
  (0, 278)	0.5905272263925467
  (0, 244)	0.36982186976548354
  (1, 1457)	0.6494640308231909
  (1, 750)	0.44626073888543205
  (1, 520)	0.5024698527821166
  (1, 233)	0.35576940936713425
  (2, 1165)	0.4973506648696112
  (2, 1163)	0.4506148851099638
  (2, 959)	0.37378798426811904
  (2, 717)	0.3824072104070653
  (2, 206)	0.31704491476442354
  (2, 80)	0.40387910535031635
  (3, 1417)	0.6574013625789735
  (3, 1045)	0.5956257031390934
  (3, 384)	0.4615771552399117
  (4, 1181)	0.4272021044641115
  (4, 567)	0.5989051065676185
  (4, 493)	0.4025799688062384
  (4, 179)	0.544738840165296
  (5, 1266)	0.2419272974625589
  (5, 950)	0.5289820811452202
  (5, 550)	0.290502405770976
  (5, 277)	0.5014298393301082
  :	:
  (2345, 1151)	0.5479599649112801
  (2345, 394)	0.31890497346203267
  (2345, 77)	0.43460807005112356
  (2346, 1450)	0.3162015309808109
  (2346, 1291)	0.5282355349125356
  (2346, 1266)	0.22894517642333365
  (2346, 867)	0.455047743879

In [33]:
len(list(tfidfvectorizer.vocabulary_.keys())),len(set(list(tfidfvectorizer.vocabulary_.keys())))

(1605, 1605)

In [34]:
tfidf_train[1124,93]

0.0

In [35]:
len(df['tfidf_tokens'].unique())

288

In [36]:
df['tfidf_tokens'].duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
339     True
340     True
341     True
342     True
343    False
Name: tfidf_tokens, Length: 344, dtype: bool

In [37]:
df.loc[df.duplicated(), :]

,tfidf_tokens,tf-idf
47,laboratory,1.000000
91,tetherless,0.776121
112,research,0.744280
113,lab,1.000000
185,infrastructure,0.777126
213,bioorganic,0.743031
217,facility,0.768427
239,lab,1.000000
263,laboratory,1.000000
281,laboratory,1.000000


In [38]:
#extract the words for with the tf-idf of higher than 0.5 for test set.
m=0
d=[]
for col in tfidf_test.nonzero()[1]:
    if tfidf_test[tfidf_train.nonzero()[0][m],col] > 0.5:
        #print (words[col], '-', tfidf_test[tfidf_train.nonzero()[0][m],col])
        d.append((tfidf_tokens[col],tfidf_test[tfidf_train.nonzero()[0][m],col]))
        m=m+1
    else:
        m=m+1
df=pd.DataFrame(d, columns=('tfidf_tokens', 'tf-idf'))
df.sort_values(by=['tf-idf'],ascending=False) 

,tfidf_tokens,tf-idf
19,canadian,0.864165
17,infrastructure,0.777126
18,systems,0.695759
14,design,0.659967
15,smart,0.644996
8,degeneration,0.643573
9,resources,0.630851
7,joint,0.585367
11,nutrition,0.580851
13,integrated,0.566357


# Model

In [39]:
x_train=tfidf_train
x_test=tfidf_test

In [40]:
x_train.shape,x_test.shape

((2350, 1605), (784, 1605))

In [41]:
y_train1.shape

(2350,)

# Model 1 Multiclass classification on x_train and y_train1 (label_total project_cost)

In [42]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)
model.fit(x_train,y_train1)

LogisticRegression(random_state=42)

In [43]:
x_test.shape,x_train.shape

((784, 1605), (2350, 1605))

In [44]:
y_test1.shape,y_train1.shape

((784,), (2350,))

In [45]:
model.score(x_train,y_train1)

0.7536170212765958

In [46]:
model.score(x_test,y_test1)

0.35331632653061223

In [47]:
y_pred=model.predict(x_test)

In [48]:
type(y_pred),type(y_test1)

(numpy.ndarray, pandas.core.series.Series)

In [49]:
y_test1.value_counts()

1    210
0    196
3    194
2    184
Name: label_total_project_costs, dtype: int64

In [50]:
pd.Series(y_pred).value_counts()

0    205
3    203
2    195
1    181
dtype: int64

In [51]:
y_preds=pd.Series(y_pred)

In [52]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test1, y_preds))

[[74 50 44 28]
 [61 54 47 48]
 [38 44 62 40]
 [32 33 42 87]]


In [54]:
#Import Seaborn
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(font_scale=1.5) # Increase font size

#def plot_conf_mat(y_test, y_preds):
  #  """
  #  Plots a confusion matrix using Seaborn's heatmap().
  #  """
  #  fig, ax = plt.subplots(figsize=(3, 3))
  #  ax = sns.heatmap(confusion_matrix(y_test, y_preds),
    #                 annot=True, # Annotate the boxes
   #                  cbar=False)
   # plt.xlabel("true label")
    #plt.ylabel("predicted label")
    
#plot_conf_mat(y_test1, y_preds)

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test1, y_preds))

              precision    recall  f1-score   support

           0       0.36      0.38      0.37       196
           1       0.30      0.26      0.28       210
           2       0.32      0.34      0.33       184
           3       0.43      0.45      0.44       194

    accuracy                           0.35       784
   macro avg       0.35      0.36      0.35       784
weighted avg       0.35      0.35      0.35       784



In [123]:
# Create a hyperparameter grid for LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
np.random.seed(42)
grid_model1= {"C": np.logspace(-4, 4, 20)}
model1 = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=grid_model1)

In [124]:
model1.fit(x_train,y_train1)

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])})

In [125]:
model1.best_params_

{'C': 0.23357214690901212}

In [126]:
model1.score(x_train,y_train1)

0.6485106382978724

In [127]:
model1.score(x_test,y_test1)

0.35586734693877553

# Multiclass classifier one vs Rest

In [56]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [57]:
clf = OneVsRestClassifier(LinearSVC())
clf.fit(x_train,y_train1)
clf.score(x_test,y_test1)

0.33035714285714285

In [58]:
y_preds1=clf.predict(x_test)
print(classification_report(y_test1, y_preds1))

              precision    recall  f1-score   support

           0       0.36      0.35      0.36       196
           1       0.25      0.22      0.24       210
           2       0.28      0.31      0.30       184
           3       0.42      0.44      0.43       194

    accuracy                           0.33       784
   macro avg       0.33      0.33      0.33       784
weighted avg       0.33      0.33      0.33       784



In [59]:
clf = OneVsRestClassifier(LinearSVC())
clf.fit(x_train,y_train2)
clf.score(x_test,y_test2)

0.34183673469387754

In [60]:
clf.score(x_train,y_train2)

0.8740425531914894

In [61]:
y_preds2=clf.predict(x_test)
print(classification_report(y_test2, y_preds2))

              precision    recall  f1-score   support

           0       0.39      0.40      0.40       230
           1       0.26      0.23      0.25       181
           2       0.30      0.29      0.29       190
           3       0.39      0.43      0.41       183

    accuracy                           0.34       784
   macro avg       0.33      0.34      0.34       784
weighted avg       0.34      0.34      0.34       784



In [128]:
np.random.seed(42)
grid_model2= {"C": np.logspace(-4, 4, 20)}
model2 = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=grid_model2)

In [129]:
model2.fit(x_train,y_train2)

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])})

In [130]:
model.score(x_train,y_train2)

0.6634042553191489

In [131]:
model.score(x_test,y_test2)

0.3482142857142857

# Regression

In [62]:
from sklearn.linear_model import LinearRegression

In [63]:
reg = LinearRegression().fit(x_train, y_train3)

In [64]:
reg.score(x_train, y_train3)

0.7143156482922549

In [65]:
reg.score(x_test, y_test3)

-2.01738308548349

In [66]:
reg = LinearRegression().fit(x_train, y_train4)
reg.score(x_train, y_train4)

0.6958067141346277

In [67]:
reg.score(x_test, y_test4)

-1.8161842160780335

In [68]:
reg = LinearRegression().fit(x_train, y_train5)
reg.score(x_train, y_train5)

0.6836284869716645

In [69]:
reg.score(x_test, y_test5)

-1.9858233712527524

In [121]:

grid_model3= {"C": np.logspace(-4, 4, 20)}
model3 = RandomizedSearchCV(LinearRegression(),
                                param_distributions=grid_model3)

In [122]:
model3.fit(x_train,y_train5)

ValueError: Invalid parameter C for estimator LinearRegression(). Check the list of available parameters with `estimator.get_params().keys()`.

# two categories prediction low and high 

In [70]:
y_train6.shape, x_train.shape


((2350,), (2350, 1605))

In [71]:
# logistic regression
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression()
logistic_model.fit(x_train, y_train6)

LogisticRegression()

In [72]:
logistic_model.score(x_train,y_train6)

0.8076595744680851

In [73]:
logistic_model.score(x_test,y_test6)

0.6020408163265306

In [74]:
y_pred = logistic_model.predict(x_test)

In [75]:
from sklearn.metrics import confusion_matrix, accuracy_score
accuracy = accuracy_score(y_test6,y_pred)*100
accuracy

60.204081632653065

In [76]:
confusion_mat = confusion_matrix(y_test6,y_pred)
confusion_mat

array([[180, 202],
       [110, 292]], dtype=int64)

In [78]:
# Create a hyperparameter grid for LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
log_reg_grid = {"C": np.logspace(-4, 4, 20)}
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid)

In [79]:
rs_log_reg.fit(x_train, y_train6)

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Atieh\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])})

In [80]:
rs_log_reg.best_params_

{'C': 11.288378916846883}

In [81]:
rs_log_reg.score(x_train, y_train6)

0.8991489361702127

In [83]:
rs_log_reg.score(x_test, y_test6)

0.5969387755102041

In [85]:
y_preds6=rs_log_reg.predict(x_test)
print(classification_report(y_test6, y_preds6))

              precision    recall  f1-score   support

           0       0.60      0.53      0.56       382
           1       0.60      0.66      0.63       402

    accuracy                           0.60       784
   macro avg       0.60      0.60      0.59       784
weighted avg       0.60      0.60      0.59       784

